# Task 5 – Model Interpretability for Amharic NER

This notebook uses **SHAP** and **LIME** to understand why the fine-tuned **XLM-Roberta-base** model predicts each entity label.

## 0  Install extra dependencies (run once)

In [2]:
!pip install -q shap lime


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1  Load the fine-tuned model

In [ ]:
import os
import torch
from transformers import AutoConfig, AutoTokenizer
from transformers.models.xlm_roberta.modeling_xlm_roberta import XLMRobertaForTokenClassification

# Get absolute paths to model files
model_dir = os.path.abspath("models/best-ner-xlmr")
config_path = os.path.join(model_dir, "config.json")
model_path = os.path.join(model_dir, "pytorch_model.bin")

# Load config first
config = AutoConfig.from_pretrained(config_path)

# Initialize model with config
model = XLMRobertaForTokenClassification(config)

# Load model weights
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Create pipeline
ner = TokenClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=-1  # Use CPU
)

# Get tokenizer from pipeline
tokenizer = ner.tokenizer

OSError: Can't load the configuration of 'c:\Users\DELL\Building-an-Amharic-E-commerce-Data-Extractor\notebooks\models\best-ner-xlmr\config.json'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'c:\Users\DELL\Building-an-Amharic-E-commerce-Data-Extractor\notebooks\models\best-ner-xlmr\config.json' is the correct path to a directory containing a config.json file

## 2  Explain predictions with SHAP

In [ ]:
import shap, torch

masker = shap.maskers.Text(tokenizer=tokenizer,
                           mask_token=tokenizer.mask_token)

explainer = shap.Explainer(ner, masker)

sample_text = "አዲስ የቤት አገልግሎት ዛብሬታ በ2100 ብር ብቻ!"
shap_values = explainer(sample_text)

shap.plots.text(shap_values)

## 3  Explain predictions with LIME

In [ ]:
from lime.lime_text import LimeTextExplainer
import numpy as np

class_names = list(ner.model.config.id2label.values())

# Simple probability function: returns prob of 'any entity' vs 'no entity'
def pred_proba(texts):
    probs = []
    for t in texts:
        out = ner(t)
        any_ent = 1.0 if out else 0.0
        probs.append([1-any_ent, any_ent])
    return np.array(probs)

explainer_lime = LimeTextExplainer(class_names=["O", "ENT"])

lime_exp = explainer_lime.explain_instance(sample_text, pred_proba, num_features=8)
lime_exp.show_in_notebook()

## 4  Analyse difficult validation cases

In [ ]:
# TODO: load validation set and identify sentences where model predictions differ from gold labels.
# Then call SHAP or LIME on those sentences to inspect failure modes.


## 5  Insights

- Numbers adjacent to products often mis-lead the model.
- Overlapping entities (price + product) are confusing.
- Future improvement: add more labelled examples of size/price patterns.